Assignment 3 notes 

In [15]:
#import packages 
import os
import pandas as pd 
import gensim.downloader as api

#packages for install
#pandas 

Flow of assignment (in one script):

- Loads the song lyric data
- Downloads/loads a word embedding model via ```gensim``` (see below)

- Takes a given word as an input and finds the most similar words via word embeddings
- Find how many songs for a given artist feature terms from the expanded query
- Calculate the percentage of that artist's songs featuring those terms
- Print and/or save results in an easy-to-understand way
    - For example, "45% of {ARTIST}'s songs contain words related to {SEARCH TERM}"

In [4]:
#load songs
filenpath = os.path.join("in", "Spotify Million Song Dataset_exported.csv")

songs = pd.read_csv(filenpath)

In [16]:
#load model
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [36]:
# Function to check if artists exists in corpus, makes both input and corpus lwoer case 
def artist_find(artist_q):
    """
    The function takes a string, if the string matches with an artist name, 
    a subset of the songs dataframe is returned, only with the given artist's songs.
    """
    artist_q = artist_q.lower()
    songs["artist"] = (songs["artist"]).str.lower()
    
    if (songs['artist'].eq(artist_q).any()):
        artist_songs = songs[songs["artist"] == artist_q]
    else: print('Artist not found') 

    return artist_songs

In [87]:
ts_songs = artist_find("Taylor Swift")

In [85]:
query = model.most_similar("love")
query_ls = ([i[0] for i in query])
query_ls.append("love")

In [133]:
ts_text = list(ts_songs["text"])

In [139]:
range(ts_text)

TypeError: 'list' object cannot be interpreted as an integer

In [93]:
song_count = 0
for text in ts_songs["text"]:
    text_tokens = text.lower().split()

    q_count = text_tokens.count(query_ls)

    if q_count > 0:
        song_count =+1
    else: pass

In [104]:
text = ts_songs["text"].iloc[0]
text_tokens = text.lower().split()

In [115]:
song_count = 0

for 
for i in query_ls:
    q_count = text_tokens.count(query_ls[i])
    if q_count> 0 :
        song_count =+1


TypeError: list.count() takes exactly one argument (0 given)

In [112]:
q_count = text_tokens.count(['you','cold'])